In [129]:
just_checking_integrity=False

In [130]:
import gc
import pandas as pd
import scipy.sparse as sps
import numpy as np

In [131]:
URM_test=sps.load_npz("../dataset/processed_data/URM_test_leaderboard.npz")

In [132]:
URM_test.shape

(4439758, 23692)

In [133]:
np.ravel(URM_test.sum(axis=1)).shape

(4439758,)

In [134]:
Session_ids=np.argwhere(np.ravel(URM_test.sum(axis=1)))

In [135]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

In [136]:
is_sorted(Session_ids)

True

In [137]:
# Create different datasets based on the selected list in order to compare between them
selected_list = ["UCF_W","ICBF_W","Graph_W","ICF_W"]

for model in selected_list:
    datasets = {}
    datasets_max = {}
#["UCF","ICBF","Graph","ICF","UCF_W","ICBF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW"]

for model in selected_list:
    df_max = None
    df_feats = None
    print(gc.collect())
    print(f"loading {model}")
    df=pd.read_csv(f"../dataset/candidates/traditional_recs/leaderboard/{model}.csv")
    print(len(df))
    print(f"loaded {model}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]

    if df_feats is not None:
        datasets[model]=df_feats
    if df_max is not None:
        datasets_max[model]=df_max
    del df
        

#del dataset_max
del df_feats
del df_max

5616
loading UCF_W
16338555
loaded UCF_W
0
loading ICBF_W
14870776
loaded ICBF_W
0
loading Graph_W
8315730
loaded Graph_W
0
loading ICF_W
17772188
loaded ICF_W


In [138]:
ids = {}
map_ids = {}
for model in selected_list:
    ids[model]=np.sort(datasets[model]["Session_Id"].unique())
    map_ids[model]=pd.DataFrame()
    map_ids[model]["Session_Id"]=ids[model]
    if just_checking_integrity:
        map_ids[model]["Real_Id"]=Session_ids[:len(ids)]
    else:
        map_ids[model]["Real_Id"]=Session_ids

In [139]:
for model in selected_list:
    datasets_max[model]=datasets_max[model].merge(map_ids[model],on="Session_Id",how="left")
    del datasets_max[model]["Session_Id"]
    datasets_max[model].rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [140]:
for model in selected_list:
    datasets[model]=datasets[model].merge(map_ids[model],on="Session_Id",how="left")
    del datasets[model]["Session_Id"]
    datasets[model].rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [141]:
import ast
submission_df_tr=pd.read_csv("../dataset/candidates/NN/Transformer/leaderboard/candidates.csv")
submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [142]:
for model in selected_list:
    datasets[model]=datasets[model].merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
del submission_df_tr

In [143]:
for model in selected_list:
    datasets[model]=datasets[model].merge(datasets_max[model],on=["Session_Id"],how="left")
    del datasets_max[model]

In [144]:
for model in selected_list:
    datasets[model].fillna(0,inplace=True)

/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_25916/4016355296.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datasets[model].fillna(0,inplace=True)


In [145]:
dataset_macro_features = pd.read_csv('../dataset/processed_data/macro_feats_NN_leaderboard.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items'])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)

for model in selected_list:
    datasets[model] = datasets[model].merge(dataset_macro_features, on='Session_Id', how='left')

In [146]:
# dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [147]:
datasets["UCF_W"].head()

,Item_ID,Score_UCF_W,Session_Id,score_transformer,rank_transformer,Max_Score_UCF_W,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,21,0.095758,26,0.002732,43.0,54.82594,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,52,0.238503,26,0.003184,36.0,54.82594,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,80,0.100885,26,0.000000,0.0,54.82594,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,91,0.000000,26,0.001053,98.0,54.82594,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,105,0.000000,26,0.001827,58.0,54.82594,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [148]:
datasets["ICBF_W"].head()

,Item_ID,Score_ICBF_W,Session_Id,score_transformer,rank_transformer,Max_Score_ICBF_W,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,9,0.491950,26,0.000000,0.0,3.397799,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,19,0.183125,26,0.000000,0.0,3.397799,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,21,0.000000,26,0.002732,43.0,3.397799,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,33,0.730614,26,0.000000,0.0,3.397799,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,52,0.000000,26,0.003184,36.0,3.397799,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [149]:
datasets["ICF_W"].head()

,Item_ID,Score_ICF_W,Session_Id,score_transformer,rank_transformer,Max_Score_ICF_W,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,3,1.350410e-02,26,0.000000,0.0,0.330278,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,9,1.882741e-22,26,0.000000,0.0,0.330278,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,21,0.000000e+00,26,0.002732,43.0,0.330278,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,33,6.852990e-03,26,0.000000,0.0,0.330278,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,52,5.488027e-01,26,0.003184,36.0,0.330278,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [150]:
datasets["Graph_W"].head()

,Item_ID,Score_Graph_W,Session_Id,score_transformer,rank_transformer,Max_Score_Graph_W,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,3,0.010701,26,0.000000,0.0,2.522003,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,21,0.019740,26,0.002732,43.0,2.522003,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,52,0.087397,26,0.003184,36.0,2.522003,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,80,0.045723,26,0.000000,0.0,2.522003,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,91,0.000000,26,0.001053,98.0,2.522003,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [151]:
gc.collect()

2200

In [152]:
for model in selected_list:
    datasets[model]=datasets[model].sort_values("Session_Id")

In [153]:
gc.collect()

0

In [154]:
import lightgbm

In [155]:
k=10

In [156]:
result = {}

In [157]:
for model_name in selected_list:
    result[model_name] = []
    for index in range(k):
        print(f"Current iteration: {str(index + 1)} of {str(k)} in model {model_name}")
        model = lightgbm.Booster(model_file=f'../rankers/model_{model_name}_{index}.txt')
        result[model_name].append(model.predict(datasets[model_name].drop(["Session_Id","Item_ID"],axis=1)))
        # dataset[f"score_{index}"]=model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1))
    
    #combine scores
    result[model_name] = np.array(result[model_name])
    result[model_name]

Current iteration: 1 of 10 in model UCF_W
Current iteration: 2 of 10 in model UCF_W
Current iteration: 3 of 10 in model UCF_W
Current iteration: 4 of 10 in model UCF_W
Current iteration: 5 of 10 in model UCF_W
Current iteration: 6 of 10 in model UCF_W
Current iteration: 7 of 10 in model UCF_W
Current iteration: 8 of 10 in model UCF_W
Current iteration: 9 of 10 in model UCF_W
Current iteration: 10 of 10 in model UCF_W
Current iteration: 1 of 10 in model ICBF_W
Current iteration: 2 of 10 in model ICBF_W
Current iteration: 3 of 10 in model ICBF_W
Current iteration: 4 of 10 in model ICBF_W
Current iteration: 5 of 10 in model ICBF_W
Current iteration: 6 of 10 in model ICBF_W
Current iteration: 7 of 10 in model ICBF_W
Current iteration: 8 of 10 in model ICBF_W
Current iteration: 9 of 10 in model ICBF_W
Current iteration: 10 of 10 in model ICBF_W
Current iteration: 1 of 10 in model Graph_W
Current iteration: 2 of 10 in model Graph_W
Current iteration: 3 of 10 in model Graph_W
Current iteratio

In [158]:
result["UCF_W"].shape

(10, 18513737)

In [159]:
for model_name in selected_list:
    datasets[model_name]["score"]=np.mean(result[model_name] ,axis=0)
# dataset["score"]=np.median(result ,axis=0)

#dataset["score"]=dataset[[f"score_{i}" for i in range(k)]].apply(lambda x:np.median(x),axis=1)

In [160]:
for model_name in selected_list:
    datasets[model_name]=datasets[model_name][["Session_Id","Item_ID","score"]]
gc.collect()

57

In [161]:
datasets["UCF_W"].head()

,Session_Id,Item_ID,score
0,26,21,-0.445097
259,26,3391,-2.332193
257,26,3389,-2.332193
256,26,3388,-2.332193
255,26,3387,-2.332193


In [162]:
datasets["ICBF_W"].head()

,Session_Id,Item_ID,score
0,26,9,-1.484400
268,26,3386,-2.326241
267,26,3385,-2.326241
266,26,3384,-2.326241
265,26,3383,-2.326241


In [163]:
datasets["ICF_W"].head()

,Session_Id,Item_ID,score
0,26,3,-1.543125
307,26,3393,-2.919423
306,26,3392,-2.919423
305,26,3391,-2.919423
304,26,3390,-2.919423


In [164]:
datasets["Graph_W"].head()

,Session_Id,Item_ID,score
0,26,3,-1.779565
131,26,3203,-1.946133
132,26,3212,-0.360857
133,26,3240,-1.557295
134,26,3266,-1.945734


In [165]:
for model_name in selected_list:
    datasets[model_name]=datasets[model_name].groupby(['Session_Id']).agg(list).reset_index()

In [166]:
def get_indices(x):
    print(f"x is {x}")
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [167]:
for model_name in selected_list:
    datasets[model_name]["indices"]=datasets[model_name]["score"].apply(lambda x: get_indices(x))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [168]:
datasets["UCF_W"].iloc[0]

Session_Id                                                   26
Item_ID       [21, 3391, 3389, 3388, 3387, 3386, 3385, 3384,...
score         [-0.44509679930495827, -2.332192549813848, -2....
indices       [279, 238, 136, 196, 202, 101, 171, 123, 350, ...
Name: 0, dtype: object

In [169]:
datasets["ICBF_W"].iloc[0]

Session_Id                                                   26
Item_ID       [9, 3386, 3385, 3384, 3383, 3379, 3373, 3372, ...
score         [-1.484400040258102, -2.32624080370014, -2.326...
indices       [287, 378, 318, 344, 211, 140, 111, 302, 116, ...
Name: 0, dtype: object

In [170]:
datasets["ICF_W"].iloc[0]

Session_Id                                                   26
Item_ID       [3, 3393, 3392, 3391, 3390, 3389, 3388, 3379, ...
score         [-1.5431252715456623, -2.9194234862297526, -2....
indices       [325, 164, 270, 239, 253, 194, 214, 435, 278, ...
Name: 0, dtype: object

In [171]:
datasets["Graph_W"].iloc[0]

Session_Id                                                   26
Item_ID       [3, 3203, 3212, 3240, 3266, 3404, 3452, 3495, ...
score         [-1.7795649561513778, -1.9461325690970075, -0....
indices       [114, 117, 59, 188, 155, 76, 149, 82, 170, 139...
Name: 0, dtype: object

In [173]:
datasets["UCF_W"].iloc[0]["Item_ID"][236]

2152

In [174]:
datasets["ICBF_W"].iloc[0]["Item_ID"][236]

1407

In [175]:
datasets["ICF_W"].iloc[0]["Item_ID"][236]

1032

In [177]:
datasets["Graph_W"].iloc[0]["Item_ID"][136]

167

In [178]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [179]:
for model_name in selected_list:
    datasets[model_name]["predictions"]=datasets[model_name][["Item_ID","indices"]].apply(get_items,axis=1)

/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_25916/744460501.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  items=np.array(x[0])
/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_25916/744460501.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return items[x[1]]
/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_25916/744460501.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  items=np.array(

In [180]:
for model_name in selected_list:
    datasets[model_name]=datasets[model_name][["Session_Id","predictions"]]
    datasets[model_name]["rank"]=datasets[model_name].apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

In [181]:
datasets["UCF_W"]

,Session_Id,predictions,rank
0,26,"[606, 1008, 4685, 1337, 1205, 4462, 3894, 4952...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"[1827, 772, 270, 3400, 3893, 4878, 3526, 3656,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"[3398, 2289, 4986, 1181, 2235, 4741, 3833, 786...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"[4148, 4418, 4686, 1726, 1139, 1609, 1136, 292...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"[3586, 1004, 3172, 143, 1165, 3180, 1518, 2485...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439446,"[4387, 2258, 4810, 3526, 2530, 2605, 2959, 386...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439458,"[3880, 3155, 2877, 3550, 3026, 2216, 3586, 135...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439550,"[4779, 4846, 625, 2266, 2431, 2638, 1901, 2448...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439653,"[3531, 42, 1015, 989, 2696, 360, 2266, 1077, 2...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [182]:
datasets["ICBF_W"]

,Session_Id,predictions,rank
0,26,"[606, 2684, 2905, 3266, 1008, 4685, 4384, 3155...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"[1827, 3400, 3893, 1331, 1449, 772, 2793, 270,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"[3398, 2289, 4986, 1181, 2235, 2075, 221, 1484...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"[2378, 3937, 4845, 1139, 4686, 3815, 4148, 441...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"[1004, 7, 25, 2873, 3550, 1518, 3705, 2485, 16...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439446,"[2258, 4387, 3782, 2959, 311, 2605, 1743, 2898...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439458,"[3026, 1248, 3880, 4770, 2705, 2919, 1875, 315...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439550,"[4779, 1565, 885, 2547, 1901, 989, 2994, 3660,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439653,"[42, 1015, 3531, 989, 1077, 3790, 1785, 2696, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [183]:
datasets["ICF_W"]

,Session_Id,predictions,rank
0,26,"[606, 4685, 1337, 1008, 1205, 3646, 3894, 2296...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"[1827, 772, 3306, 4878, 3400, 3893, 270, 1449,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"[3398, 2289, 4986, 1181, 2235, 2075, 1039, 384...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"[3337, 2378, 4111, 3872, 1815, 794, 4845, 3815...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"[1004, 3172, 2175, 1165, 3705, 3586, 3309, 25,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439446,"[2258, 4387, 3084, 3782, 3867, 4810, 2605, 174...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439458,"[2877, 3880, 3026, 2705, 1248, 4406, 2919, 187...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439550,"[1901, 4779, 1565, 885, 989, 517, 2431, 4575, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439653,"[3531, 42, 1015, 360, 3665, 2266, 989, 2696, 4...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [184]:
datasets["Graph_W"]

,Session_Id,predictions,rank
0,26,"[606, 1008, 4685, 1337, 1205, 4462, 426, 3894,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"[1827, 772, 3400, 3893, 3229, 270, 1449, 3656,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"[3398, 2289, 4986, 1181, 2235, 914, 4931, 2075...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"[2378, 1726, 1139, 4686, 4418, 4148, 1609, 415...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"[3172, 1165, 3586, 1004, 25, 1518, 3180, 3309,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439446,"[2258, 4387, 4810, 3526, 2605, 1468, 3782, 386...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439458,"[3880, 2877, 3155, 3026, 2216, 1110, 3550, 358...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439550,"[4779, 4846, 625, 1901, 2431, 2266, 1565, 2638...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439653,"[3531, 42, 1015, 989, 2696, 360, 1077, 4148, 4...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [185]:
for model_name in selected_list:
    datasets[model_name]["predictions"]=datasets[model_name]["predictions"].apply(tuple)

In [186]:
for model_name in selected_list:
    datasets[model_name]=datasets[model_name][["Session_Id","predictions","rank"]]

In [187]:
for model_name in selected_list:
    datasets[model_name].columns=["session_id","code","rank"]

In [188]:
datasets["UCF_W"].head()

,session_id,code,rank
0,26,"(606, 1008, 4685, 1337, 1205, 4462, 3894, 4952...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"(1827, 772, 270, 3400, 3893, 4878, 3526, 3656,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"(3398, 2289, 4986, 1181, 2235, 4741, 3833, 786...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"(4148, 4418, 4686, 1726, 1139, 1609, 1136, 292...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"(3586, 1004, 3172, 143, 1165, 3180, 1518, 2485...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [190]:
datasets["ICBF_W"].head()

,session_id,code,rank
0,26,"(606, 2684, 2905, 3266, 1008, 4685, 4384, 3155...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"(1827, 3400, 3893, 1331, 1449, 772, 2793, 270,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"(3398, 2289, 4986, 1181, 2235, 2075, 221, 1484...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"(2378, 3937, 4845, 1139, 4686, 3815, 4148, 441...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"(1004, 7, 25, 2873, 3550, 1518, 3705, 2485, 16...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [191]:
datasets["ICF_W"].head()

,session_id,code,rank
0,26,"(606, 4685, 1337, 1008, 1205, 3646, 3894, 2296...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"(1827, 772, 3306, 4878, 3400, 3893, 270, 1449,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"(3398, 2289, 4986, 1181, 2235, 2075, 1039, 384...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"(3337, 2378, 4111, 3872, 1815, 794, 4845, 3815...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"(1004, 3172, 2175, 1165, 3705, 3586, 3309, 25,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [192]:
datasets["Graph_W"].head()

,session_id,code,rank
0,26,"(606, 1008, 4685, 1337, 1205, 4462, 426, 3894,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"(1827, 772, 3400, 3893, 3229, 270, 1449, 3656,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"(3398, 2289, 4986, 1181, 2235, 914, 4931, 2075...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"(2378, 1726, 1139, 4686, 4418, 4148, 1609, 415...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"(3172, 1165, 3586, 1004, 25, 1518, 3180, 3309,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [194]:
submission_df = {}
for model_name in selected_list:
    submission_df[model_name] = datasets[model_name].explode(['code', 'rank']).astype(int)

In [195]:
mapping= pd.read_csv('../dataset/processed_data/map.csv')

for model_name in selected_list:
    submission_df[model_name]=submission_df[model_name].merge(mapping,on="code",how="left")
    del submission_df[model_name]["code"]
    submission_df[model_name]=submission_df[model_name][["session_id","item_id","rank"]]

In [198]:
print(len(submission_df["UCF_W"]))

5000000


In [199]:
submission_df["UCF_W"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 3 columns):
 #   Column      Dtype
---  ------      -----
 0   session_id  int64
 1   item_id     int64
 2   rank        int64
dtypes: int64(3)
memory usage: 114.4 MB


In [200]:
submission_df["UCF_W"].isna().sum()

session_id    0
item_id       0
rank          0
dtype: int64

In [201]:
for model_name in selected_list:
    submission_df[model_name].to_csv(
        f'../leaderboard_sub_{model_name}.csv',
        index=False
        )

In [210]:
test_sessions= pd.read_csv("../dataset/original_data/test_leaderboard_purchases.csv")
#test_sessions_full=pd.read_csv("../dataset/processed_data/test_leaderboard_sessions_full_mapped.csv")
#test_sessions= pd.concat([test_sessions, test_sessions_full])

In [211]:
test_sessions.head()

,session_id,item_id,date
0,26,5383,2021-06-16 09:54:52.696
1,200,27613,2021-06-25 12:24:56.31
2,205,15246,2021-06-11 00:29:03.381
3,495,4009,2021-06-14 22:15:43.818
4,521,742,2021-06-19 13:52:31.149


In [212]:
final_comprasion_df = {}

for model_name in selected_list:
    final_comprasion_df[model_name] = pd.merge(test_sessions, submission_df[model_name], on=["session_id", "item_id"], how="left").fillna(100)

In [213]:
final_comprasion_df["UCF_W"].head()

,session_id,item_id,date,rank
0,26,5383,2021-06-16 09:54:52.696,2.0
1,200,27613,2021-06-25 12:24:56.31,6.0
2,205,15246,2021-06-11 00:29:03.381,100.0
3,495,4009,2021-06-14 22:15:43.818,100.0
4,521,742,2021-06-19 13:52:31.149,4.0


In [214]:
mrr_dict = {}

In [215]:
for model_name in selected_list:
    mrr_dict[model_name] = (1 / final_comprasion_df[model_name].groupby('session_id')['rank'].min()).mean()

In [216]:
print(mrr_dict)

{'UCF_W': 0.1943128374154572, 'ICBF_W': 0.1780241736897335, 'Graph_W': 0.19314809218632945, 'ICF_W': 0.18835764532640784}


In [219]:
compression_table = pd.DataFrame([mrr_dict.values()], index=['MRR@100'], columns=[f"{x}+Transformer" for x in mrr_dict.keys()])

In [220]:
compression_table

,UCF_W+Transformer,ICBF_W+Transformer,Graph_W+Transformer,ICF_W+Transformer
MRR@100,0.194313,0.178024,0.193148,0.188358
